# Web Scrapping the List of People With Access to PBI Reports

**Author:** Cristian C. Velandia C.

**Description:** This notebook shows how to configure Selenium Chrome Driver and perform a simple web scarpping task to create an inventory in excel about people with access to a Power BI report, This was done this way given the fact that Power BI does not offer a simple way to extract those lists, not even for admins. This offers a simple and effective solution to thtat problematic. 

**Input:** csv file with the following columns with no headers
* company: The name of the company that belongs to the report
* Working Area: Power BI work area name, This could be extracted during the process but for simplicity while extracting the URLs can be added to the list 
* PBI Report Name: Report name, This could be extracted during the process but for simplicity while extracting the URLs can be added to the list 
* URL: The URL of the Direct Access section of the Power BI Report

**Output:** An Excel File that contains the list of people with access and the permissions. 

**Main Libraries to be used:** 
- selenium                  4.16.0
- pandas                    2.1.1
- openpyxl                  3.1.2

**Python Version:** Python 3.9.18

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

import time 
import os 
from os.path import exists
import pandas as pd
from datetime import date

In [ ]:
# Create folder for chrome driver data
Folder = "Chrome_Driver_Data"
# Parent Directory path
Directorio = "C:\\"
#create folder path 
path_chrome = os.path.join(Directorio, Folder)

CHECK_FOLDER = os.path.isdir(path_chrome)

# If folder doesn't exist, then create it
if not CHECK_FOLDER:
    os.makedirs(path_chrome)

# Define the driver to use
chromeOptions = webdriver.ChromeOptions()
# add extra configuration options to the driver 
chromeOptions.add_argument("--start-maximized") 
#chromeOptions.add_argument("--window-size=1300,200")
chromeOptions.add_argument("--disable-gpu") 
chromeOptions.add_experimental_option("excludeSwitches", ["enable-logging"]) 
chromeOptions.add_argument("user-data-dir=" + path_chrome) 
chromeOptions.add_argument("--homepage=about:blank") #start on blank page to avoid wasting time

#Automatically download the driver using the service method from chrome
service = Service()
driver = webdriver.Chrome(service = service, options = chromeOptions)

In [ ]:
# Read URLs to all the power BI Direct access page
enlaces = pd.read_csv("enlaces tableros_nata.txt", header=None)

In [ ]:
# Cicle to access each URL in the csv file (LOGIN manually before running this cell)

for index, row in enlaces.iterrows():

    URL = row[3] #Extract URL
    entidad = row[0] #Extract compnay name 
    area = row[1] #Extract working area
    tablero = row[2] #Extract pbi report name 

    driver.get(URL) #Send the URL to the chrome driver

    print(tablero) # Print to know wht is being processed

    time.sleep(8) # Wait for webpage to load

    accesos = [] # Empty list for storing the file

    # Scroll down 20 times while searching for each line in the table of access
    for i in range(20): 
        for j in range(1, 51, 1):
            try:
                element = driver.find_element(By.XPATH, '//*[@id="artifactContentView"]/div[1]/div[{0}]'.format(j)) #Find element of the table
                accesos.append(element.text) #append element to the list
                #print('//*[@id="artifactContentView"]/div[1]/div[{0}]'.format(j))
            except:
                #break if element not found
                #print("break")
                break
            
        scroll_origin = ScrollOrigin.from_element(element) # scroll wodn from last element found
        ActionChains(driver).scroll_from_origin(scroll_origin, 0, 1500).perform() # scroll 1500 pixels
        time.sleep(1)
    
    # Process Scraped data with pandas
    accesos_df = pd.DataFrame(accesos) #Create dataframe based on the list
    accesos_df_final = accesos_df[0].str.split("\n", expand = True).copy() #split text to create a table
    accesos_df_final.rename(columns = {0: "iniciales", 1:"Persona", 2:"Correo", 3:"Permisos"}, inplace=True)
    accesos_df_final.drop_duplicates(inplace=True) #drop duplicates generate in scrolling
    accesos_df_final = accesos_df_final.apply(lambda x: x.shift(periods=1) if x["Permisos"] is None else x, axis = 1) # Shift cells right for rows without intials
    
    accesos_df_final["Entidad"] = entidad
    accesos_df_final["Area"] = area
    accesos_df_final["Tablero"] = tablero
    accesos_df_final["Url"] = URL
    id = str(index) # add index as id of the access
    accesos_df_final.to_excel("{0}_accesos {1} {2} {3} {4}.xlsx".format(id, entidad, area, tablero, str(date.today()))) #Save file to Excel

In [ ]:
# Close selenium Driver
driver.quit()